<a href="https://colab.research.google.com/github/mlaricobar/solution-datathon-interbank-2020/blob/desarrollo/plus-one-solution/%5B04%5D%20Model%20Building%20-%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONSTRUCCIÓN DE LOS MODELOS


### Instalación de Librerías necesarias

In [1]:
!pip install catboost

     |████████████████████████████████| 65.8MB 47kB/s 



### Lectura de Datos

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/'My Drive'/Competencias/'[02] Interbank-2020'

/gdrive/My Drive/Competencias/[02] Interbank-2020


In [4]:
!ls -lh '[01] Intermediate'

total 4.0G
-rw------- 1 root root 521M Dec 31 22:53 ds_rcc.csv
-rw------- 1 root root  62M Jan 16 01:23 ds_rcc_feat_combination_test.csv
-rw------- 1 root root  56M Jan 16 01:23 ds_rcc_feat_combination_train.csv
-rw------- 1 root root 719M Jan 13 03:51 ds_rcc_feat_hist_test.csv
-rw------- 1 root root 645M Jan 13 03:49 ds_rcc_feat_hist_train.csv
-rw------- 1 root root 185M Dec 18 16:51 ds_rcc_instit.csv
-rw------- 1 root root 204M Dec 18 16:51 ds_rcc_instit_test.csv
-rw------- 1 root root 284M Jan 16 02:24 ds_rcc_riesgo.csv
-rw------- 1 root root 314M Jan 16 02:25 ds_rcc_riesgo_test.csv
-rw------- 1 root root 576M Dec 31 22:55 ds_rcc_test.csv
-rw------- 1 root root  22M Dec 31 22:28 ds_se.csv
-rw------- 1 root root  24M Dec 31 22:28 ds_se_test.csv
-rw------- 1 root root  18M Dec 31 19:34 ds_sunat.csv
-rw------- 1 root root  20M Dec 31 19:34 ds_sunat_test.csv
-rw------- 1 root root  16K Jan 16 01:24 overfit_per_feature.csv
-rw------- 1 root root 192M Dec 31 13:48 rcc_product_level.csv
-r

In [5]:
import numpy as np
import pandas as pd

In [6]:
def optimizar_df(df):
  
  for col_ in df.select_dtypes(include=[int]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="unsigned")

  for col_ in df.select_dtypes(include=[float]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="float")
  
  return df

In [7]:
df_base_train = optimizar_df(pd.read_csv("[00] Data/y_train.csv"))
df_se = optimizar_df(pd.read_csv('[01] Intermediate/ds_se.csv'))
df_rcc = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc.csv"))
df_rcc_2 = optimizar_df(pd.read_csv("[01] Intermediate/rcc_product_level.csv"))
df_rcc_hist = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_feat_hist_train.csv"))
df_rcc_comb = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_feat_combination_train.csv"))
df_rcc_riesgo = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_riesgo.csv"))
#df_censo_train = optimizar_df(pd.read_csv("[00] Data/censo_train.csv"))#.drop_duplicates()
#df_sunat_train = optimizar_df(pd.read_csv("[01] Intermediate/ds_sunat.csv"))

df_base_test = optimizar_df(pd.read_csv("[00] Data/sample_submission.csv"))
df_se_test = optimizar_df(pd.read_csv('[01] Intermediate/ds_se_test.csv'))
df_rcc_test = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_test.csv"))
df_rcc_test_2 = optimizar_df(pd.read_csv("[01] Intermediate/rcc_product_level_test.csv"))
df_rcc_test_hist = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_feat_hist_test.csv"))
df_rcc_test_comb = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_feat_combination_test.csv"))
df_rcc_test_riesgo = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_riesgo_test.csv"))
#df_censo_test = optimizar_df(pd.read_csv("[00] Data/censo_test.csv"))#.drop_duplicates()
#df_sunat_test = optimizar_df(pd.read_csv("[01] Intermediate/ds_sunat_test.csv"))

In [8]:
#df_rcc_2.loc[df_rcc_2["key_value"].isin([19746, 21193,  23473,  38585,  60175,  62839])]

In [25]:
df_se = df_se[["key_value", "edad", "sexo", "est_cvl", "sit_lab", "flg_sin_email", "tip_lvledu"]]
df_se_test = df_se_test[["key_value", "edad", "sexo", "est_cvl", "sit_lab", "flg_sin_email", "tip_lvledu"]]

#### Consolidar Dataset

In [26]:
df_train = (df_base_train
                 .merge(df_rcc, how="left", on=["key_value"])
                 .merge(df_rcc_2, how="left", on=["key_value"])
                 .merge(df_rcc_hist, how="left", on=["key_value"])
                 .merge(df_rcc_comb, how="left", on=["key_value"])
                 .merge(df_rcc_riesgo, how="left", on=["key_value"])
                 .merge(df_se, how="left", on=["key_value"])
                 #.merge(df_sunat_train, how="left", on=["key_value"])
                 #.merge(df_censo_train, how="left", on=["key_value"])
                .reset_index(drop=True)
           )#.merge(df_rcc_inst, how="left", on=["key_value"])#.merge(df_rcc, how="left", on=["key_value"])
            #.merge(df_sunat_train, how="left", on=["key_value"])
            #.merge(df_censo_train, how="left", on=["key_value"])

df_test = (df_base_test
                 .merge(df_rcc_test, how="left", on=["key_value"])
                 .merge(df_rcc_test_2, how="left", on=["key_value"])
                 .merge(df_rcc_test_hist, how="left", on=["key_value"])
                 .merge(df_rcc_test_comb, how="left", on=["key_value"])
                 .merge(df_rcc_test_riesgo, how="left", on=["key_value"])
                 .merge(df_se_test, how="left", on=["key_value"])
                 #.merge(df_sunat_test, how="left", on=["key_value"])
                 #.merge(df_censo_test, how="left", on=["key_value"])
                 .reset_index(drop=True)
           )#.merge(df_rcc_inst_test, how="left", on=["key_value"])#.merge(df_rcc_test, how="left", on=["key_value"])
           #.merge(df_sunat_test, how="left", on=["key_value"])
           #.merge(df_censo_test, how="left", on=["key_value"])

In [27]:
del df_train["target_y"]
del df_test["target_y"]

In [28]:
df_train = df_train.rename(columns={"target_x": "target"})
df_test = df_test.rename(columns={"target_x": "target"})

In [29]:
df_train.head()

,key_value,target,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_255_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_255_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_255_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,...,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_1_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_2_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_2_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_2_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_3_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_3_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_3_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_4_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_4_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_4_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_0_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_0_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_0_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_2_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_2_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_2_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_3_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_3_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_3_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_4_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_4_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_4_COUNT,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_MEAN,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_SUM,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_COUNT,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_MEAN,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_SUM,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_COUNT,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_MEAN,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_SUM,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_COUNT,U12M_RECENCY_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR,U12M_LONG_RECENCY_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR,U12M_DIFF_MONTHS_FIRST_LAST_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR,edad,sexo,est_cvl,sit_lab,flg_sin_email,tip_lvledu
0,0,0,1.062248,3.186743,3,0.927645,0.927645,1,0.961819,2.885458,3,0.961558,2.884674,3,NaN,0.000000,0,NaN,0.000000,0,0.927880,0.927880,1,NaN,0.0,0,NaN,0.0,0,NaN,0.000000,0,NaN,0.0,0,0.986265,2.958796,3,3.604134,10.812402,...,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.973117,11.677406,12,1

In [ ]:
df_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358487 entries, 0 to 358486
Columns: 982 entries, key_value to U12M_DIFF_MONTHS_FIRST_LAST_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR
dtypes: float32(712), float64(10), uint16(11), uint32(3), uint8(246)
memory usage: 1.1 GB


In [ ]:
df_test.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396666 entries, 0 to 396665
Columns: 982 entries, key_value to U12M_DIFF_MONTHS_FIRST_LAST_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR
dtypes: float32(716), float64(151), uint16(7), uint32(3), uint8(105)
memory usage: 1.6 GB


In [30]:
for col_ in df_train.columns:
  if df_train.loc[df_train[col_] == np.inf, :].shape[0] > 0:
    print(col_)

for col_ in df_test.columns:
  if df_test.loc[df_test[col_] == np.inf, :].shape[0] > 0:
    print(col_)

In [31]:
avoid_columns = ["U12M_codmes_last", "U12M_codmes_first"] #+ ['cod_ubi', 'prv', 'edad', 'dto', 'SUNATFECALTA_MEAN']
main_columns = ["key_value", "target"]
predictores = [c for c in df_train.columns if c not in (main_columns + avoid_columns)]

In [32]:
print(len(predictores), predictores)

970 ['U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_255_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_255_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_255_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_MEAN', '

### Preparar Marco de Evaluación

In [33]:
nrows, nrows_train, nrows_test = (df_train.shape[0] + df_test.shape[0]), df_train.shape[0], df_test.shape[0]

print("Cantidad de filas: {0:,}\n\t- Train:\t{1:,} filas\t({2:}%)\n\t- Test:\t\t{3:,} filas\t({4:}%)".format(nrows,
                                                                                                            nrows_train,
                                                                                                            round((100 * (nrows_train/nrows)), 2),
                                                                                                            nrows_test,
                                                                                                            round((100 * (nrows_test/nrows)), 2)
                                                                                                            ))

Cantidad de filas: 755,153
	- Train:	358,487 filas	(47.47%)
	- Test:		396,666 filas	(52.53%)


In [34]:
df_train.head()

,key_value,target,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_255_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_255_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_255_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,...,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_1_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_2_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_2_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_2_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_3_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_3_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_3_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_4_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_4_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_3_4_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_0_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_0_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_0_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_2_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_2_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_2_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_3_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_3_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_3_COUNT,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_4_MEAN,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_4_SUM,U12M_SALDO_MEAN_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR_5_4_COUNT,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_MEAN,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_SUM,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_COUNT,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_MEAN,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_SUM,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_COUNT,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_MEAN,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_SUM,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECTO_COD_CLAS_DEUDOR_COUNT,U12M_RECENCY_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR,U12M_LONG_RECENCY_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR,U12M_DIFF_MONTHS_FIRST_LAST_FOR_RIESGO_DIRECTO_COD_CLAS_DEUDOR,edad,sexo,est_cvl,sit_lab,flg_sin_email,tip_lvledu
0,0,0,1.062248,3.186743,3,0.927645,0.927645,1,0.961819,2.885458,3,0.961558,2.884674,3,NaN,0.000000,0,NaN,0.000000,0,0.927880,0.927880,1,NaN,0.0,0,NaN,0.0,0,NaN,0.000000,0,NaN,0.0,0,0.986265,2.958796,3,3.604134,10.812402,...,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.973117,11.677406,12,1

### Construcción de Modelos

In [35]:
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

from scipy.optimize import differential_evolution

In [36]:
seed_val = 41

ctb_params = {'loss_function':'Logloss',
              'eval_metric':'AUC',
              'task_type': 'GPU',
              'verbose': 200,
              'random_seed': 1
             }

lgb_params = {
                "objective": "binary",
                "metric": "auc",
                "max_depth": 8,
                "learning_rate": 0.05,
                "bagging_seed": 41,
                "min_data_in_leaf": 20,
                "bagging_fraction": 0.7,
                "feature_fraction": 0.7,
                "bagging_freq": 5,
                "verbosity": -1
              }

In [37]:
k = 5 # n folds

def model_with_cv(df_train, df_test, target_class, predictores, df_val=None):

  ##################################################################
  ############# ETAPA DE PREDICCIÓN
  ##################################################################

  # Se instancian las variables que se obtendrán como resultado.
  results_models = []
  list_models_cv = []

  df_probs_train = df_train[["key_value", target_class]].copy().reset_index(drop=True)
  df_probs_test = df_test[["key_value"]].copy().reset_index(drop=True)

  if df_val is not None:
    df_probs_val = df_val[["key_value", target_class]].copy().reset_index(drop=True)
  else:
    df_probs_val = None
  # Generar los Folds
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

  for i, (traincv, valcv) in enumerate(skf.split(df_train[predictores], df_train[target_class])):

      print("Iteracion {}".format(i))
      
      df_train_cv = df_train.loc[traincv, :].copy().reset_index(drop=True)
      df_val_cv = df_train.loc[valcv, :].copy().reset_index(drop=True)

      # Fase de Entrenamiento
      print("\t CLASIFICACIÓN")
      print("\t\t 1. CATBOOST")
      ## CatBoost
      cbt_model = CatBoostClassifier(**ctb_params)
      cbt_model.fit(df_train_cv[predictores], df_train_cv[target_class], eval_set=(df_val_cv[predictores], df_val_cv[target_class]), use_best_model=True)

      print("\t\t 2. LIGHTGBM")
      ## LightGBM
      lgtrain = lgb.Dataset(df_train_cv[predictores], label=df_train_cv[target_class])
      lgval = lgb.Dataset(df_val_cv[predictores], label=df_val_cv[target_class])
      lgb_model = lgb.train(lgb_params, lgtrain, 900, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=20, evals_result={})

      # Fase de Scoring
      ## Clasificación
      probas_train_cbtm = cbt_model.predict_proba(df_train_cv[predictores])[:, 1]
      probas_val_cbtm = cbt_model.predict_proba(df_val_cv[predictores])[:, 1]

      probas_train_lgbm = lgb_model.predict(df_train_cv[predictores])
      probas_val_lgbm = lgb_model.predict(df_val_cv[predictores])

      # Cálculo de Métricas
      ## Clasificación
      metric_train_cbtm = roc_auc_score(df_train_cv[target_class], probas_train_cbtm)
      metric_val_cbtm = roc_auc_score(df_val_cv[target_class], probas_val_cbtm)

      metric_train_lgbm = roc_auc_score(df_train_cv[target_class], probas_train_lgbm)
      metric_val_lgbm = roc_auc_score(df_val_cv[target_class], probas_val_lgbm)

      if df_val is not None:
        probas_val_out_cbtm = cbt_model.predict_proba(df_val[predictores])[:, 1]
        metric_val_out_cbtm = roc_auc_score(df_val[target_class], probas_val_out_cbtm)

        probas_val_out_lgbm = lgb_model.predict(df_val[predictores])
        metric_val_out_lgbm = roc_auc_score(df_val[target_class], probas_val_out_lgbm)

      else:
        probas_val_out_cbtm = np.zeros((1,1))
        metric_val_out_cbtm = 0

        probas_val_out_lgbm = np.zeros((1,1))
        metric_val_out_lgbm = 0

      # Cálculo de probabilidades en la base de Train y Validacion
      df_probs_train.loc[valcv, "prob_cbtm"] = probas_val_cbtm
      df_probs_train.loc[valcv, "prob_lgbm"] = probas_val_lgbm
      df_probs_train.loc[valcv, "FOLD"] = "FOLD_{0}".format(i+1)

      
      df_probs_test["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_test[predictores])[:, 1]
      df_probs_test["prob_lgbm_{}".format(i)] = lgb_model.predict(df_test[predictores])

      if df_val is not None:
        df_probs_val["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_val[predictores])[:, 1]
        df_probs_val["prob_lgbm_{}".format(i)] = lgb_model.predict(df_val[predictores])

        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, "auc_val_out_cbtm": metric_val_out_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, "auc_val_out_lgbm": metric_val_out_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4), round(df_val[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4), round(np.mean(probas_val_out_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4), round(np.std(probas_val_out_cbtm),4)))

      else:

        # Almacenar los resultados
        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4)))
        
        
      list_models_cv.append({"cbtm_model": cbt_model, "lgbm_model": lgb_model})
      
      print("CATBOOST REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_cbtm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_cbtm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)

      print("LIGHTGBM REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_lgbm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_lgbm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)
  
  ##################################################################
  ############# PARA ETAPA DE OPTIMIZACIÓN
  ##################################################################

  if df_val is not None:
    df_probs_val["prob_avg_cbtm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_cbtm" in c]].mean(axis=1)
    df_probs_val["prob_avg_lgbm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_lgbm" in c]].mean(axis=1)

  df_probs_test["prob_avg_cbtm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_cbtm" in c]].mean(axis=1)
  df_probs_test["prob_avg_lgbm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_lgbm" in c]].mean(axis=1)

  
  return results_models, list_models_cv, df_probs_train, df_probs_val, df_probs_test

In [38]:
results, list_models, df_probs_train, df_probs_val, df_probs_test = model_with_cv(df_train, df_test, "target", predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.044018
0:	learn: 0.7462732	test: 0.7440774	best: 0.7440774 (0)	total: 23.2ms	remaining: 23.2s
200:	learn: 0.8412845	test: 0.8360178	best: 0.8360178 (200)	total: 4.4s	remaining: 17.5s
400:	learn: 0.8491512	test: 0.8412477	best: 0.8412477 (400)	total: 8.6s	remaining: 12.9s
600:	learn: 0.8536621	test: 0.8432536	best: 0.8432536 (600)	total: 12.8s	remaining: 8.48s
800:	learn: 0.8571885	test: 0.8444423	best: 0.8444423 (800)	total: 16.9s	remaining: 4.19s
999:	learn: 0.8602655	test: 0.8452091	best: 0.8452091 (999)	total: 21s	remaining: 0us
bestTest = 0.8452090621
bestIteration = 999
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.82279	valid_1's auc: 0.816966
[40]	training's auc: 0.833253	valid_1's auc: 0.826468
[60]	training's auc: 0.841083	valid_1's auc: 0.833104
[80]	training's auc: 0.846479	valid_1's auc: 0.837365
[100]	training's auc: 0.850571	valid_1's auc: 0.839967
[120

In [ ]:
results, list_models, df_probs_train, df_probs_val, df_probs_test = model_with_cv(df_train, df_test, "target", predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.044018
0:	learn: 0.7627705	test: 0.7609657	best: 0.7609657 (0)	total: 24.4ms	remaining: 24.4s
200:	learn: 0.8416596	test: 0.8361103	best: 0.8361103 (200)	total: 4.5s	remaining: 17.9s
400:	learn: 0.8499506	test: 0.8416783	best: 0.8416783 (400)	total: 8.7s	remaining: 13s
600:	learn: 0.8546715	test: 0.8437790	best: 0.8437790 (600)	total: 12.8s	remaining: 8.48s
800:	learn: 0.8586056	test: 0.8452917	best: 0.8452976 (798)	total: 16.9s	remaining: 4.19s
999:	learn: 0.8618044	test: 0.8461689	best: 0.8461689 (999)	total: 21s	remaining: 0us
bestTest = 0.8461688757
bestIteration = 999
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.821775	valid_1's auc: 0.81574
[40]	training's auc: 0.833252	valid_1's auc: 0.826639
[60]	training's auc: 0.841207	valid_1's auc: 0.833358
[80]	training's auc: 0.847155	valid_1's auc: 0.837859
[100]	training's auc: 0.851398	valid_1's auc: 0.840592
[120]	

In [ ]:
from sklearn.model_selection import cross_validate

ctb_params = {'loss_function':'Logloss',
              'eval_metric':'AUC',
              'task_type': 'GPU',
              'verbose': False,
              'random_seed': 1
             }

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)
cbt_model = CatBoostClassifier(**ctb_params)

def get_auc_scores(top_n):
  cbt_metrics_dict = cross_validate(cbt_model, df_train[predictores_ordered[:top_n]] , df_train["target"], scoring=["roc_auc"], cv=skf, return_train_score=True)
  return {"train_roc_auc": cbt_metrics_dict["train_roc_auc"].mean(), "test_roc_auc": cbt_metrics_dict["test_roc_auc"].mean()}

In [ ]:
import time

In [ ]:
l = []
for i in range(len(predictores)):
  print(i)
  ti = time.time()
  auc_score_dict = get_auc_scores(i+1)
  auc_score_dict["predictor"] = predictores_ordered[i]
  tf = time.time()
  auc_score_dict["timing"] = round(tf - ti, 2)
  l.append(auc_score_dict)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
df_predictor_scores = pd.DataFrame(l)

In [ ]:
df_predictor_scores

,train_roc_auc,test_roc_auc,predictor,timing
0,0.593561,0.593558,sit_lab,46.23
1,0.629363,0.627519,edad,51.68
2,0.655921,0.650343,cod_ubi,52.11
3,0.660335,0.652235,prv,52.28
4,0.664777,0.655792,sexo,51.56
...,...,...,...,...
285,0.756936,0.746884,gas_otrs_e_18,78.13
286,0.756866,0.746893,gas_esp_a_18,78.65
287,0.756892,0.746908,gas_esp_e_18,78.96
288,0.756915,0.746878,gas_ali_b_18,78.05


In [ ]:
df_predictor_scores.to_csv('[01] Intermediate/overfit_per_feature.csv')

In [ ]:
df_predictor_scores = optimizar_df(pd.read_csv("[01] Intermediate/overfit_per_feature.csv"))
df_predictor_scores['diff'] = df_predictor_scores['train_roc_auc'] - df_predictor_scores['test_roc_auc']
del df_predictor_scores['Unnamed: 0']
df_predictor_scores['prev_diff'] = df_predictor_scores["diff"].shift(1)
df_predictor_scores['overfit_power'] = df_predictor_scores["diff"] - df_predictor_scores['prev_diff']

In [ ]:
df_predictor_scores.sort_values(by="overfit_power", ascending=False).head(50)

,train_roc_auc,test_roc_auc,predictor,timing,diff,prev_diff,overfit_power
2,0.655921,0.650343,cod_ubi,52.110001,0.005578,0.001844,0.003734
3,0.660335,0.652235,prv,52.279999,0.008101,0.005578,0.002523
1,0.629363,0.627519,edad,51.680000,0.001844,0.000002,0.001842
9,0.679589,0.666507,dto,53.740002,0.013082,0.011671,0.001411
10,0.682753,0.668558,SUNATFECALTA_MEAN,53.029999,0.014195,0.013082,0.001113
4,0.664777,0.655792,sexo,51.560001,0.008986,0.008101,0.000885
8,0.678079,0.666407,cod_ocu,53.070000,0.011671,0.010835,0.000837
5,0.666379,0.656619,rgn,52.139999,0.009760,0.008986,0.000775
7,0.675818,0.664983,tip_lvledu,51.590000,0.010835,0.010160,0.000675
13,0.686775,0.671380,ubigeo,53.820000,0.015394,0.014766,0.000629


In [ ]:
100 * ((df_predictor_scores["train_roc_auc"] - df_predictor_scores["test_roc_auc"])/ df_predictor_scores["test_roc_auc"])

0     0.000389
1     0.289922
2     0.852185
3     1.236606
4     1.344005
5     1.446230
6     1.522647
7     1.789403
8     1.868896
9     1.962733
10    1.940254
11    1.905266
12    1.926305
13    1.946367
dtype: float64

In [ ]:
len(predictores_ordered)

14

In [ ]:
pd.DataFrame(cbt_metrics_dict)

,fit_time,score_time,test_roc_auc,train_roc_auc
0,9.234891,0.070340,0.648054,0.656113
1,9.319264,0.082008,0.654158,0.655115
2,9.312224,0.070756,0.646339,0.656984
3,9.279877,0.070521,0.654439,0.654955
4,9.470471,0.084299,0.648917,0.656336


In [ ]:
pd.DataFrame(cbt_metrics_dict)

,fit_time,score_time,test_roc_auc,train_roc_auc
0,9.690796,0.085083,0.670414,0.688688
1,9.624147,0.077374,0.677919,0.687217
2,9.600421,0.076942,0.669380,0.689437
3,9.442307,0.074758,0.682003,0.686598
4,9.603130,0.073854,0.674694,0.688036


In [ ]:
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.697757,0.671606,0.717521,0.674016,41
1,0.696393,0.679365,0.707314,0.681183,41
2,0.698447,0.671620,0.714218,0.672443,41
3,0.695538,0.683119,0.714596,0.684847,41
4,0.696629,0.675931,0.710205,0.676935,41


In [ ]:
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.754771,0.741972,0.770223,0.739408,41
1,0.755850,0.738341,0.767508,0.736977,41
2,0.755494,0.738280,0.772519,0.737204,41
3,0.754723,0.744403,0.789787,0.744341,41
4,0.756476,0.736167,0.793822,0.734685,41


In [ ]:
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.764330,0.749673,0.779308,0.748610,41
1,0.765868,0.746725,0.781919,0.745780,41
2,0.765939,0.746301,0.812963,0.746141,41
3,0.764067,0.753698,0.783674,0.752913,41
4,0.766307,0.744788,0.784620,0.743226,41


In [ ]:
results, list_models, df_probs_train, df_probs_val, df_probs_test = model_with_cv(df_train, df_test, "target", predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.044018
0:	learn: 0.7450264	test: 0.7452695	best: 0.7452695 (0)	total: 19ms	remaining: 19s
200:	learn: 0.8381600	test: 0.8315840	best: 0.8315840 (200)	total: 3.3s	remaining: 13.1s
400:	learn: 0.8471795	test: 0.8372967	best: 0.8372967 (400)	total: 6.54s	remaining: 9.77s
600:	learn: 0.8528716	test: 0.8399592	best: 0.8399601 (599)	total: 9.76s	remaining: 6.48s
800:	learn: 0.8572438	test: 0.8414835	best: 0.8414835 (800)	total: 13s	remaining: 3.22s
999:	learn: 0.8609880	test: 0.8427463	best: 0.8427463 (999)	total: 16.1s	remaining: 0us
bestTest = 0.8427462876
bestIteration = 999
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.816188	valid_1's auc: 0.809385
[40]	training's auc: 0.82754	valid_1's auc: 0.820189
[60]	training's auc: 0.835966	valid_1's auc: 0.827433
[80]	training's auc: 0.842149	valid_1's auc: 0.832389
[100]	training's auc: 0.84673	valid_1's auc: 0.835348
[120]	tr

In [ ]:
# First Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.878254,0.864993,0.927719,0.866271,41
1,0.878423,0.864160,0.927449,0.866562,41
2,0.878053,0.864858,0.920229,0.866549,41
3,0.877606,0.867833,0.919273,0.869289,41
4,0.878562,0.864014,0.919363,0.864704,41


In [ ]:
# First Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.834539,0.822712,0.859006,0.823846,41
1,0.835161,0.820945,0.855816,0.821616,41
2,0.835441,0.817861,0.857260,0.819050,41
3,0.835025,0.821256,0.851961,0.822491,41
4,0.835012,0.819481,0.872939,0.821429,41


In [ ]:
# Second Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.843651,0.829637,0.890753,0.831454,41
1,0.844369,0.827960,0.869049,0.829013,41
2,0.844213,0.825024,0.880458,0.827586,41
3,0.843544,0.828033,0.883655,0.830802,41
4,0.843915,0.827735,0.890578,0.830337,41


In [ ]:
# Third Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.837454,0.822779,0.875155,0.824367,41
1,0.837709,0.822735,0.871554,0.823324,41
2,0.838256,0.819201,0.881905,0.820629,41
3,0.837436,0.822897,0.872295,0.824793,41
4,0.837011,0.822719,0.881190,0.824380,41


In [ ]:
# Fourth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.848285,0.833916,0.898653,0.836538,41
1,0.848607,0.832712,0.890521,0.834790,41
2,0.848699,0.830145,0.891555,0.832881,41
3,0.848064,0.831962,0.897827,0.835895,41
4,0.848851,0.831197,0.898466,0.833766,41


In [ ]:
# Fifth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.854373,0.842748,0.890797,0.844431,41
1,0.855238,0.840793,0.892966,0.842644,41
2,0.855101,0.836793,0.903635,0.840386,41
3,0.854775,0.839483,0.902767,0.842496,41
4,0.855069,0.839889,0.903772,0.841770,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.949859,0.948399,0.968543,0.966217,41
1,0.949407,0.947514,0.968718,0.966161,41
2,0.949645,0.947987,0.968701,0.965941,41
3,0.949299,0.948137,0.968743,0.966143,41
4,0.949288,0.947315,0.968823,0.966365,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.860083,0.848422,0.892553,0.849547,41
1,0.860588,0.845172,0.896773,0.846512,41
2,0.860836,0.841588,0.903166,0.845084,41
3,0.860436,0.845567,0.910398,0.848112,41
4,0.860556,0.845273,0.900377,0.847654,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.860826,0.849093,0.894307,0.850258,41
1,0.861321,0.845266,0.894554,0.847733,41
2,0.861743,0.842110,0.912972,0.845521,41
3,0.861320,0.845963,0.912363,0.848544,41
4,0.861374,0.845834,0.912110,0.848061,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.878833,0.870579,0.918745,0.871590,41
1,0.879513,0.865097,0.918189,0.866995,41
2,0.879361,0.864071,0.920135,0.865900,41
3,0.879635,0.865029,0.928408,0.867146,41
4,0.879792,0.866854,0.928434,0.869142,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.860988,0.842746,0.904525,0.845485,41
1,0.860294,0.846048,0.911042,0.848863,41
2,0.860974,0.843292,0.903123,0.845444,41
3,0.859557,0.847849,0.892344,0.849564,41
4,0.860571,0.844696,0.899653,0.846400,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.862629,0.845316,0.916792,0.847565,41
1,0.862215,0.847788,0.915426,0.850558,41
2,0.862449,0.845452,0.905724,0.847748,41
3,0.861513,0.849538,0.894507,0.851724,41
4,0.862641,0.846382,0.897622,0.848652,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.862310,0.845926,0.891823,0.848227,41
1,0.861702,0.848007,0.904694,0.851192,41
2,0.862279,0.846104,0.907136,0.848737,41
3,0.861353,0.849822,0.902781,0.852547,41
4,0.862098,0.846543,0.906835,0.848319,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.861804,0.846169,0.915858,0.849037,41
1,0.861105,0.848644,0.902668,0.852051,41
2,0.861333,0.845930,0.904123,0.849078,41
3,0.860733,0.850149,0.905612,0.852847,41
4,0.861367,0.846743,0.891866,0.849184,41


### Importancia de Variables

In [39]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models[0]["cbtm_model"].get_feature_importance()})
for i in range(1, 5):
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models[i]["cbtm_model"].get_feature_importance()}), how="left", on="name")

df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)
df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(30)

,name,importance_fold_0,importance_fold_1,importance_fold_2,importance_fold_3,importance_fold_4,importance_folds_mean
0,long_recency_mean,6.138281,6.856750,6.787107,6.898809,6.606887,6.657567
1,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,2.542841,2.402913,2.470441,2.234583,2.141055,2.358367
2,tip_lvledu,2.276504,2.439708,2.254113,2.281031,2.236996,2.297670
3,U12M_saldo_count_last,1.969712,2.550868,2.071765,1.329995,2.251008,2.034670
4,REAL_PRODUCT_COUNT_STD_FOR_RIESGO_DIRECTO_COD_...,1.869933,1.749779,2.318008,1.955535,2.115930,2.001837
5,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECT...,1.790955,1.549545,2.048863,2.246481,2.097503,1.946669
6,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_RIESGO_DIRECT...,1.951987,2.172196,1.735397,2.038556,1.802563,1.940140
7,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,1.544175,2.037203,1.978599,1.798652,2.084615,1.888649
8,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,1.915448,1.818600,1.906674,1.735720,2.029301,1.881149
9,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,1.317306,2.010660,1.841432,1.966941,1.925930,1.812454


In [ ]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models[0]["cbtm_model"].get_feature_importance()})
for i in range(1, 5):
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models[i]["cbtm_model"].get_feature_importance()}), how="left", on="name")

df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)
df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(30)

,name,importance_fold_0,importance_fold_1,importance_fold_2,importance_fold_3,importance_fold_4,importance_folds_mean
0,U12M_condicion_max_last,6.093806,6.374718,5.856256,6.141838,6.292357,6.151795
1,long_recency_mean,5.871285,6.171751,5.952338,5.915460,6.748272,6.131821
2,U12M_condicion_sum_last,2.378633,2.622469,2.373150,2.998815,3.217341,2.718082
3,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,2.439944,1.875343,2.718732,2.364312,2.389584,2.357583
4,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,2.117087,2.464258,2.106777,1.888657,2.211893,2.157735
5,tip_lvledu,2.113853,2.013019,2.082510,2.008059,2.054614,2.054411
6,REAL_PRODUCT_COUNT_STD_FOR_RIESGO_DIRECTO_COD_...,1.993197,1.531749,1.982588,1.854724,1.934500,1.859352
7,U12M_saldo_count_last,2.024622,1.949893,2.038096,1.859673,1.274503,1.829357
8,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,1.505270,1.842358,1.314621,1.368102,2.508613,1.707793
9,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,1.867784,1.677258,1.776804,1.600820,1.580669,1.700667


In [ ]:
df_predictores_.shape[0]

290

In [ ]:
predictores_ordered = df_predictores_["name"].tolist()

### Estabilidad de las Variables

In [ ]:
def func_var_porc(n, d):
  try:
    return round(100.0 * (n/d - 1), 2)
  except:
    return None

In [ ]:
df_predictores_["train_mean"] = df_predictores_["name"].apply(lambda c: df_train[c].mean())
df_predictores_["train_std"] = df_predictores_["name"].apply(lambda c: df_train[c].std())
df_predictores_["test_mean"] = df_predictores_["name"].apply(lambda c: df_test[c].mean())
df_predictores_["test_std"] = df_predictores_["name"].apply(lambda c: df_test[c].std())

df_predictores_["var_porc_mean_test_train"] = df_predictores_.apply(lambda row: func_var_porc(row["test_mean"], row["train_mean"]), axis=1)
df_predictores_["var_porc_std_test_train"] = df_predictores_.apply(lambda row: func_var_porc(row["test_std"], row["train_std"]), axis=1)

df_predictores_["abs_var_porc_mean_test_train"] = abs(df_predictores_["var_porc_mean_test_train"])

In [ ]:
df_predictores_.shape, df_predictores_.loc[df_predictores_["abs_var_porc_mean_test_train"] < 10].shape

((677, 14), (362, 14))

In [ ]:
new_predictores = df_predictores_.loc[df_predictores_["abs_var_porc_mean_test_train"] < 10, "name"].tolist()

### Interpretación

### Generando Submit

In [ ]:
df_probs_test[["key_value", "prob_avg_cbtm"]].rename(columns={"prob_avg_cbtm": "target"}).to_csv('[02] Output/submit_03.csv', index=False)

In [ ]:
df_probs_test.shape

(396666, 13)

In [ ]:
!ls -lh '[02] Output'

ls: cannot access '[02] Output': No such file or directory


------